# Calculation of betas

## 1.0 Introduction

The walkability index integrates both physical and perception-based variables. The physical variables represent the characteristics of the built environment that support pedestrian activity along a street segment. In contrast, the perception variables capture how the population evaluates and prioritizes the conditions of those segments. Our study considers six variables—population density, intersection density, land use mix, vegetation, slope, and sidewalks—each with a physical and a perception component.

The perception data were obtained through surveys conducted in the study areas of Medellín and Guadalajara. The survey questions were designed to assess whether pedestrians perceived each variable as having a positive influence on their chosen walking routes. Responses were recorded on a Likert scale, with qualitative options ranging from strongly disagree to strongly agree, and were subsequently transformed into quantitative values on a 0-to-1 scale, where 0 corresponds to strongly disagree and 1 to strongly agree.

In addition to evaluating the perceived influence of each physical variable on a specific route segment, the survey also captured the relative importance participants assigned to each factor when deciding whether to walk. This dual approach allows the index to reflect not only how the built environment is experienced, but also how much each element matters in pedestrian decision-making.

## 2.0 Libraries

To interact with the database and process the various data, we have created a python script, `analysis.py`, which stores the different functions required for the task. In addition, other Python libraries are also used for data analysis and visualization.

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder

import os
import sys

module_path = os.path.abspath(os.path.join('../'))
if module_path not in sys.path:
    sys.path.append(module_path)
import src.analysis as an

c:\Users\Laura\anaconda3\envs\ox_eafit\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 3.0 Initial data

The survey data collected in the study areas must be cleaned to retain only the relevant qualitative responses provided by the population.

In [2]:
# Address of the file xlsx
medellin_csv = "../data/output/table/encuestas/Encuesta caminata (Medellín)(1-613).xlsx"
# Reading the file 
df_medellin = pd.read_excel(medellin_csv)
# Delete some wrong survyes
df_medellin = df_medellin[df_medellin['Encuestador']!='Valentina_Vasquez']
# Filter the survyes by used equipment and disability
df_medellin = df_medellin[df_medellin['¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?'] == 'Si']
df_medellin = df_medellin[df_medellin['¿Se encuentra usted en alguna condición de discapacidad?'] != 'Si']
# Eliminate some unnecesary columns
df_medellin = df_medellin.drop(columns=['Hora de inicio', 'Correo electrónico', 'Nombre', 'Hora de la última modificación',
                    'Fecha de realización', 'Encuestador', 'Encuesta #', 'Hora de finalización', 
                    'Lugar donde se realizó la encuesta',
                    'Lugar donde se realizó la encuesta (Moravia)', 'Lugar donde se realizó la encuesta (Floresta)', 
                    'Lugar donde se realizó la encuesta (Aguacatala)',
                    '¿Hace uso de algún equipamiento/sitio de interés localizado en el sector/territorio?', ##
                    '¿Cómo suple las necesidades de equipamientos educativos?', ##
                    '¿Cómo suple las necesidades de equipamientos de cultura o deportivos?', ##
                    '¿Se encuentra usted en alguna condición de discapacidad?', ###
                    'Indique cuál de las siguientes condiciones de discapacidad presenta', ###
                    'Indique el lugar donde inició de su recorrido lo más preciso posible (Punto de referencia, intersección más cercana)', 
                    'Indique la hora de salida de su lugar de origen de recorrido (formato militar)', 'Ingrese los minutos de salida', 
                    'Indique el lugar de destino de su recorrido lo más precios posible (Punto de referencia, intersección más cercana) (old)',
                    'Indique el lugar de destino de su recorrido',
                    '¿Cuánto tiempo le tomó el recorrido caminando para legar a su lugar de destino? (Ingresar en minutos)', 
                    'Describa la ruta que tomó para este último viaje desde su lugar de origen hasta su lugar de destino. Puede utilizar direcciones y/o lugares conocidos de referencia para facilitar la identificación...'], ###
                               axis=1)
# Change the name of some columns
nombres_nuevos = {
    'Estrato socioeconómico (NO APLICA)': 'Estrato socioeconómico',
    'La cercanía del equipamiento X': 'Qué tan importante es para usted la cercanía del equipamiento X',
    'La presencia de andenes': 'Qué tan importante es para usted la presencia de andenes',
    'Sentirme seguro/a frente a otros vehículos': 'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos',
    'Sentirme seguro/a frente a posibles delitos en mi trayecto': 'Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto',
    'La posibilidad de hacer múltiples vueltas/trámites en mi recorrido': 'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido',
    'La posibilidad de poder tomar desvíos y hacer múltiples trayectos en mi recorrido': 'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido',
    'Sentirme cómodo/a por la sombra generada por la vegetación': 'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación',
    'Sentirme cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido': 'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
    'Que el paisaje sea agradable': 'Qué tan importante es para usted que el paisaje sea agradable',
    'Column': '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?',
    '2': '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?',
    '3': '38. ¿Se te hace largo y aburrido el recorrido?',
    '4': '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?',
    '5': '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?',
    '6': '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?',
    '7': '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?',
    '8': '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?',
    '9': '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?',
    '10': '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?',
    '11': '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?',
    '12': '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?',
    '¿Cuáles?': '¿Cuáles equipamientos de interés en el sector utilizas?'
}
df_medellin.rename(columns=nombres_nuevos, inplace=True)
# Division between walking and non walking survyes
med_caminantes = df_medellin[df_medellin["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] == "Caminata"]
med_no_caminantes = df_medellin[df_medellin["¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)"] != "Caminata"]
# Delete columns that don't alling with walking survyes
med_caminantes = med_caminantes.drop(columns=['¿Qué medio de transporte utilizó para llegar al equipamiento X? (último viaje realizado)',
                                            'En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando el mismo modo de transporte?',
                                            'Debido a la lejanía del equipamiento X', 'Debido a la ausencia de andenes',
                                            'Porque no me siento seguro/a frente a otros vehículos', 
                                            'Porque no me siento seguro/a frente a posibles delitos en mi trayecto', 
                                            'Debido al esfuerzo que debo hacer a causa de la pendiente de las calles', 
                                            'Debido a que no me siento cómodo/a por el calor', 'Debido a que siento que los andenes muy saturados de gente', 
                                            'Debido a que el paisaje a lo largo del recorrido no es agradable'], axis=1)
# Transform qualitative answers to cuantitatuve
lista_columnas = ["Género", "Edad (años)", "Estrato socioeconómico", "Nivel educativo completado", "En una semana típica, ¿Cuántas veces realiza este mismo viaje utilizando la caminata como modo de transporte? (CAMINATA)"]
orden = [['Femenino','Masculino'], 
         ['10-19', '20-29', '30-39', '40-49', '50-59', 'Mayor a 60'], 
         ['E1', 'E2', 'E3', 'E4', 'E5', 'E6'], 
         ['Ninguno', 'Primaria', 'Secundaria', 'Técnico', 'Tecnológico', 'Pregrado', 'Posgrado'], 
         ['Muy poco frecuente', '1', '2', '3', '4', '5 o más']]
for i, columna in enumerate(lista_columnas):
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden[i])
med_caminantes = med_caminantes.rename(columns={'Edad (años)': 'Rango edad (10 años)'})
med_caminantes = med_caminantes.rename(columns={'¿Cuáles?': '¿Cuáles equipamientos de interés en el sector utilizas?'})

## 4.0 Conversion of survey responses for quantitative analysis

### 4.1 Perception variables

To enable quantitative analysis, the qualitative responses collected through the Likert scale were converted into numerical values. Each option on the scale, originally ranging from strongly disagree to strongly agree, was mapped onto a continuous scale from 0 to 1, where 0 represents the lowest level of agreement and 1 the highest. This transformation facilitates the integration of perception variables into the walkability index and supports comparative and statistical analyses.

In [3]:
# Select the perception and questions
columnas_percepcion = ['36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?', 
                       '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?', 
                       '38. ¿Se te hace largo y aburrido el recorrido?', 
                       '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?', 
                       '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?', 
                       '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?', 
                       '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?', 
                       '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?', 
                       '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?', 
                       '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?', 
                       '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?', 
                       '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?']
med_caminantes[columnas_percepcion].head()

,"36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?","37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?",38. ¿Se te hace largo y aburrido el recorrido?,"39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?","40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?","41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?","42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?","43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?","44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?","45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?","46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?","47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?"
0,De acuerdo,De acuerdo,En desacuerdo,En desacuerdo,De acuerdo,En desacuerdo,En desacuerdo,De acuerdo,De acuerdo,En desacuerdo,De acuerdo,De acuerdo
2,"Ni de acuerdo, ni en desacuerdo",De acuerdo,Totalmente en desacuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,"Ni de acuerdo, ni en desacuerdo",De acuerdo,De acuerdo,Totalmente en desacuerdo,Totalmente de acuerdo,Totalmente de acuerdo
3,Totalmente de acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,En desacuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",Totalmente en desacuerdo,De acuerdo,De acuerdo
5,De acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,De acuerdo,Totalmente de acuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,De acuerdo,Totalmente en desacuerdo,De acuerdo,En desacuerdo
7,De acuerdo,"Ni de acuerdo, ni en desacuerdo",En desacuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo","Ni de acuerdo, ni en desacuerdo",En desacuerdo,En desacuerdo,Totalmente en desacuerdo,En desacuerdo,"Ni de acuerdo, ni en desacuerdo"


In [4]:
# Define the order of the categories, from lower to higher
# (The number is assign as follows: 0 to the first element, 1 to the second, 2 to the third, and so on)
orden = ['Totalmente en desacuerdo', 'En desacuerdo', 'Ni de acuerdo, ni en desacuerdo', 'De acuerdo', 'Totalmente de acuerdo']
for columna in columnas_percepcion:
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden)
    med_caminantes[columna] = med_caminantes[columna]*0.25 # Multiplied for 0.25 to match the range [0-1]
    med_caminantes[columna] = pd.to_numeric(med_caminantes[columna])
med_caminantes[columnas_percepcion].head()

,"36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?","37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?",38. ¿Se te hace largo y aburrido el recorrido?,"39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?","40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?","41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?","42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?","43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?","44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?","45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?","46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?","47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?"
0,0.75,0.75,0.25,0.25,0.75,0.25,0.25,0.75,0.75,0.25,0.75,0.75
2,0.50,0.75,0.00,0.25,0.50,0.75,0.50,0.75,0.75,0.00,1.00,1.00
3,1.00,0.50,0.25,0.75,1.00,0.75,0.25,0.25,0.50,0.00,0.75,0.75
5,0.75,0.50,0.25,0.75,1.00,0.25,0.50,0.25,0.75,0.00,0.75,0.25
7,0.75,0.50,0.25,0.25,0.50,0.50,0.50,0.25,0.25,0.00,0.25,0.50


For certain survey items where higher raw responses indicated a less favorable impact on route choice (i.e., a negative effect on walkability), the scores were transformed using the expression 1 – response. This transformation was applied to standardize the directionality of all variables, ensuring that higher values consistently represent a more favorable contribution to perceived walkability.

In [5]:
med_caminantes['38. ¿Se te hace largo y aburrido el recorrido?'] = 1 - med_caminantes['38. ¿Se te hace largo y aburrido el recorrido?']
med_caminantes = med_caminantes.rename(columns={'38. ¿Se te hace largo y aburrido el recorrido?': '38. ¿Se te hace corto y agradable el recorrido?'})
med_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'] = 1 - med_caminantes['41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?']
med_caminantes = med_caminantes.rename(columns={'41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?': '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar NO se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?'})
med_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'] = 1 - med_caminantes['45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?']
med_caminantes = med_caminantes.rename(columns={'45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar te implicó un gran esfuerzo físico debido a la pendiente/inclinación?': '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar NO te implicó un gran esfuerzo físico debido a la pendiente/inclinación?'})

### 4.2 Importance variables

The importance variables—representing general perceptions of what factors influence walkability—were estimated following the same procedure used for the perception variables.

In [6]:
# Select the importance questions
columnas_importancia = ['Qué tan importante es para usted la cercanía del equipamiento X', 
                       'Qué tan importante es para usted la presencia de andenes', 
                       'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos', 
                       'Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto', 
                       'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido', 
                       'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido', 
                       'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación', 
                       'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido', 
                       'Qué tan importante es para usted que el paisaje sea agradable']
# Define the order of the categories, from lower to higher
# (The number is assign as follows: 0 to the first element, 1 to the second, 2 to the third, and so on)
orden = ['No es importante', 'Es poco importante', 'Moderadamente importante', 'Importante',   'Muy importante']
    
for columna in columnas_importancia:
    med_caminantes = an.ordinal_encoding(med_caminantes, columna, orden)
    med_caminantes[columna] = med_caminantes[columna]*0.25
    med_caminantes[columna] = pd.to_numeric(med_caminantes[columna])

## 5.0 Betas estimation

Our study considers six variables that influence the walkability of a territory: population density, intersection density, land use mix, vegetation, slope, and sidewalks. For these variables, twelve survey questions were designed to capture their perceived impact on walking decisions. To consolidate the perception data, we calculated the mean response for each variable, resulting in a single representative perception value per variable.

In parallel, the importance of these factors—defined as the degree to which respondents consider each variable relevant for walkability—was assessed through nine survey questions. It is important to note that no importance score was estimated for the slope variable, as the survey design did not include a question addressing its perceived importance.

In [ ]:
columnas_percepcion = [# Question for population density
                       '36. A lo largo de su recorrido, ¿Sientes que hay personas a tu alrededor que pudieran apoyarte en caso de algún problema?', 
                       # Questions for intersection density
                       '37. A lo largo de su recorrido, ¿Sientes que hay diversas intersecciones que te permiten cruzar las calles de manera segura?', 
                       '38. ¿Se te hace corto y agradable el recorrido?', 
                       '39. A lo largo de tu recorrido, ¿Es posible desviarse en múltiples puntos/intersecciones para cambiar de ruta?', 
                       # Questions for land use mix
                       '40. A lo largo de tu recorrido, ¿Puedes aprovechar para acceder a múltiples servicios y satisfacer necesidades diversas (realizar vueltas/pendientes, comprar alimentos, divertirse, etc.)?', 
                       '41. A lo largo de tu recorrido, ¿Sientes que la comodidad en tu caminar NO se ve influenciada por la cantidad de personas y actividades que se realizan en el andén?', 
                       '42. A lo largo de tu recorrido, ¿Encuentras elementos que hacen que tu caminata sea poco monótona?', 
                       # Questions for vegetation
                       '43. A lo largo de tu recorrido, ¿Sientes que el tipo de vegetación te genera sombra y protección del sol?', 
                       '44. A lo largo de tu recorrido, ¿Sientes que la presencia y el tipo de vegetación crea un entorno agradable?', 
                       # Question for slope
                       '45. A lo largo de tu recorrido, ¿Consideras que el hecho de caminar NO te implicó un gran esfuerzo físico debido a la pendiente/inclinación?', 
                       # Questions for sidewalk
                       '46. A lo largo de tu recorrido, ¿Te sientes seguro(a) ante la presencia de vehículos?', 
                       '47. A lo largo de tu recorrido, ¿Sientes que la continuidad de andenes te permite caminar de manera cómoda (no hay: cambios de niveles, interrupciones repentinas del andén, presencia de obstáculos, cambios en el ancho del andén, etc.)?'
                       ]
columnas_importancia = [# Question for population density
                        'Qué tan importante es para usted sentirse seguro/a frente a posibles delitos en su trayecto',
                        # Questions for intersection density
                        'Qué tan importante es para usted la posibilidad de poder tomar desvíos y hacer múltiples trayectos en su recorrido', 
                        # Questions for land use mix
                        'Qué tan importante es para usted la posibilidad de hacer múltiples vueltas/trámites en su recorrido',
                        'Qué tan importante es para usted sentirse cómodo/a en términos de la cantidad de gente a lo largo de mi recorrido',
                        # Questions for vegetation
                        'Qué tan importante es para usted sentirse cómodo/a por la sombra generada por la vegetación', 
                        'Qué tan importante es para usted que el paisaje sea agradable',    
                        # Questions for sidewalk
                        'Qué tan importante es para usted sentirse seguro/a frente a otros vehículos', 
                        'Qué tan importante es para usted la presencia de andenes',
                        # Questions for closeness (although it is asked, it is not considered for the calculation of betas)
                        'Qué tan importante es para usted la cercanía del equipamiento X'
                        ]
TAs = ['TA_Moravia','TA_Floresta','TA_Aguacatala']
TA = TAs[0]
# Filter by the area of analysis
TA_caminantes = med_caminantes[med_caminantes["Territorio de análisis"] == TA]
# Filter by questions
TA_caminantes_percepcion = TA_caminantes[columnas_percepcion]
TA_caminantes_importancia = TA_caminantes[columnas_importancia]
# Segmentation of perception questions
TA_per_densidad_pob = TA_caminantes_percepcion[columnas_percepcion[0]]
TA_per_densidad_inter = TA_caminantes_percepcion[columnas_percepcion[1:4]]
TA_per_usos_suelo = TA_caminantes_percepcion[columnas_percepcion[4:7]]
TA_per_ind_vegetacion = TA_caminantes_percepcion[columnas_percepcion[7:9]]
TA_per_pendiente = TA_caminantes_percepcion[columnas_percepcion[9]]
TA_per_presenc_aceras = TA_caminantes_percepcion[columnas_percepcion[10:]]
# Segmentation of importance questions
TA_imp_densidad_pob = TA_caminantes_importancia[columnas_importancia[0]]
TA_imp_densidad_inter = TA_caminantes_importancia[columnas_importancia[1]]
TA_imp_usos_suelo = TA_caminantes_importancia[columnas_importancia[2:4]]
TA_imp_ind_vegetacion = TA_caminantes_importancia[columnas_importancia[4:6]]
TA_imp_cercania = TA_caminantes_importancia[columnas_importancia[8]]
TA_imp_presenc_aceras = TA_caminantes_importancia[columnas_importancia[6:8]]
# Estimation of perception beta coefficients for physical variables
TA_per_campos = [TA_per_densidad_pob, TA_per_densidad_inter, TA_per_usos_suelo, TA_per_ind_vegetacion, TA_per_pendiente, TA_per_presenc_aceras]
lista = ['densidad_pob', 'densidad_inter', 'usos_suelo', 'ind_vegetacion', 'pendiente', 'presenc_aceras']
TA_per_betas_campos = {}
for i in range(len(TA_per_campos)):
    TA_per_betas_campos[lista[i]] = TA_per_campos[i].mean().mean()
# Resume
df_betas = pd.DataFrame(list(TA_per_betas_campos.items()), columns=['variable', 'beta']).round(4)
change = {'densidad_pob':'den_pop','densidad_inter':'den_inter','usos_suelo':'land_use',
        'ind_vegetacion':'ind_ndvi','pendiente':'grade_abs','presenc_aceras':'sidewalk'}
df_betas = df_betas.replace({'variable':change})
print(TA, 'perception')
print(df_betas)
# Estimation of importance beta coefficients for physical variables
TA_imp_campos = [TA_imp_densidad_pob, TA_imp_densidad_inter, TA_imp_usos_suelo, TA_imp_ind_vegetacion, TA_imp_presenc_aceras]
lista_imp = ['densidad_pob', 'densidad_inter', 'usos_suelo', 'ind_vegetacion', 'presenc_aceras']
TA_imp_betas_campos = {}
for i in range(len(TA_imp_campos)):
    TA_imp_betas_campos[lista_imp[i]] = TA_imp_campos[i].mean().mean()
# Resume
df_betas_imp = pd.DataFrame(list(TA_imp_betas_campos.items()), columns=['variable', 'beta']).round(4)
change = {'densidad_pob':'den_pop','densidad_inter':'den_inter','usos_suelo':'land_use',
        'ind_vegetacion':'ind_ndvi','presenc_aceras':'sidewalk'}
df_betas_imp = df_betas_imp.replace({'variable':change})
print(TA, 'importance')
print(df_betas_imp)

TA_Moravia perception
    variable    beta
0    den_pop  0.6813
1  den_inter  0.6708
2   land_use  0.5865
3   ind_ndvi  0.6326
4  grade_abs  0.7032
5   sidewalk  0.6059
TA_Moravia importance
    variable    beta
0    den_pop  0.6546
1  den_inter  0.5935
2   land_use  0.6393
3   ind_ndvi  0.7176
4   sidewalk  0.7214
